# Data Collection Pipeline V4 - Pareto Top-N Approach

## Two-Phase Approach:

### Phase 1: Build Cost Pool
- Generate random cost parameters
- Optimize each with ODL API
- Store (costs, features) pairs
- Can skip if you already have cost_pool.json

### Phase 2: Match Preferences (New!)
- Generate random preferences
- Get **top 30 solutions** from pool (closest to ideal)
- Use **Pareto evaluation** to select best
- Save training samples

## Benefits:
- ✅ 30 unique outputs (vs 5-7 with old method)
- ✅ High quality (top 15% of solutions)
- ✅ No arbitrary scaling factors
- ✅ Ready for regression training

## 1. Imports

In [1]:
import json
import time
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import copy
from typing import List, Dict

print("✓ Imports successful")

✓ Imports successful


## 2. Configuration

**⚠️ UPDATE THESE VALUES:**

In [2]:
# API Configuration


# Model file (use relaxed version without skill requirements)
MODEL_PATH = "../../test_model_mathias_fixed.json"  # ← CHANGE THIS

# Phase 1: Cost Pool Generation
POOL_SIZE = 200  # Number of cost sets to generate
WAIT_SECONDS = 30  # Wait time for optimization (can be 30-120s)
CHECKPOINT_EVERY = 10  # Save progress every N samples
POOL_FILE = "cost_pool.json"  # Output file

# Phase 2: Training Data Generation  
N_SAMPLES = 400  # Number of training samples to generate
TOP_N = 30  # Use top N solutions from pool (30 = good diversity)
OUTPUT_FILE = "training_data_v4_pareto.json"  # Output file

# Cost parameter ranges
COST_RANGES = {
    'costPerTravelHour': (0.5, 5.0),
    'costPerKm': (0.01, 0.5),
    'parking_multiplier': (0.2, 2.0)
}

print(f"Configuration loaded:")
print(f"  Pool size: {POOL_SIZE}")
print(f"  Training samples: {N_SAMPLES}")
print(f"  Top-N selection: {TOP_N}")
print(f"  Estimated Phase 1 time: {(POOL_SIZE * WAIT_SECONDS) / 3600:.1f} hours")

Configuration loaded:
  Pool size: 200
  Training samples: 400
  Top-N selection: 30
  Estimated Phase 1 time: 1.7 hours


## 3. Pareto Analyzer Class

In [3]:
class ParetoAnalyzer:
    """Top-N + Pareto selection for cost pool matching"""
    
    def __init__(self, cost_pool: List[Dict]):
        self.cost_pool = cost_pool
        self.objectives = ['avg_parking_difficulty', 'total_travel_hours', 'total_distance_km']
    
    def get_top_n_solutions(self, n=30) -> List[Dict]:
        """
        Get top N solutions by distance to ideal point (0,0,0)
        
        These are the N "best" solutions overall, regardless of preferences.
        """
        if len(self.cost_pool) == 0:
            return []
        
        # Extract features for all solutions
        all_features = np.array([[s['features'][obj] for obj in self.objectives] 
                                for s in self.cost_pool])
        
        # Normalize to [0, 1] range
        min_vals = all_features.min(axis=0)
        max_vals = all_features.max(axis=0)
        normalized = (all_features - min_vals) / (max_vals - min_vals + 1e-10)
        
        # Calculate distance to ideal point (0,0,0)
        distances = np.sqrt(np.sum(normalized**2, axis=1))
        
        # Select top N by lowest distance
        n = min(n, len(self.cost_pool))  # Don't exceed pool size
        best_indices = np.argsort(distances)[:n]
        
        return [self.cost_pool[i] for i in best_indices]
    
    def normalize_objectives(self, solutions: List[Dict]) -> np.ndarray:
        """Normalize objectives to [0, 1] for fair comparison"""
        if len(solutions) == 0:
            return np.array([])
        
        values = np.array([[s['features'][obj] for obj in self.objectives] 
                          for s in solutions])
        
        normalized = np.zeros_like(values, dtype=float)
        for j in range(len(self.objectives)):
            min_val = values[:, j].min()
            max_val = values[:, j].max()
            if max_val > min_val:
                normalized[:, j] = (values[:, j] - min_val) / (max_val - min_val)
            else:
                normalized[:, j] = 0.0
        
        return normalized
    
    def select_best(self, solutions: List[Dict], preferences: Dict) -> Dict:
        """
        Select best solution from list using weighted Euclidean distance
        
        Args:
            solutions: List of candidate solutions
            preferences: User preference weights
        
        Returns:
            Best matching solution (copy with added metadata)
        """
        if len(solutions) == 0:
            return None
        
        if len(solutions) == 1:
            result = solutions[0].copy()
            result['pareto_distance'] = 0.0
            result['selection_pool_size'] = 1
            return result
        
        # Normalize objectives
        normalized = self.normalize_objectives(solutions)
        
        # Create weight vector from preferences
        weights = np.array([
            preferences['parking_importance'],
            preferences['time_importance'],
            preferences['distance_importance']
        ])
        
        # Calculate weighted Euclidean distance to ideal (0,0,0)
        distances = np.sqrt(np.sum((normalized * weights) ** 2, axis=1))
        
        # Select solution with minimum distance
        best_idx = np.argmin(distances)
        
        # Return copy with metadata
        result = solutions[best_idx].copy()
        result['pareto_distance'] = float(distances[best_idx])
        result['selection_pool_size'] = len(solutions)
        
        return result

print("✓ ParetoAnalyzer class defined")

✓ ParetoAnalyzer class defined


## 4. Data Collector Class

In [4]:
class CostPoolDataCollector:
    """Two-phase data collector with Pareto Top-N selection"""
    
    def __init__(self, base_url, username, password, model_path, cost_ranges):
        self.base_url = base_url.rstrip('/')
        self.username = username
        self.password = password
        self.cost_ranges = cost_ranges
        
        # Initialize session
        self.session = requests.Session()
        self.session.auth = (username, password)
        self.session.headers.update({
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        })
        
        # Load base model
        with open(model_path, 'r') as f:
            self.base_model = json.load(f)
        
        print(f"✓ Initialized")
        print(f"  Jobs: {len(self.base_model['data']['jobs'])}")
        print(f"  Vehicles: {len(self.base_model['data']['vehicles'])}")
        
        # Data storage
        self.cost_pool = []
        self.training_data = []
    
    # ========================================================================
    # HELPER METHODS
    # ========================================================================
    
    def sample_random_costs(self):
        """Sample random cost parameters"""
        costs = {}
        for param, (min_val, max_val) in self.cost_ranges.items():
            costs[param] = np.random.uniform(min_val, max_val)
        
        # Round for cleaner values
        costs['costPerTravelHour'] = round(costs['costPerTravelHour'], 2)
        costs['costPerKm'] = round(costs['costPerKm'], 4)
        costs['parking_multiplier'] = round(costs['parking_multiplier'], 2)
        
        return costs
    
    def sample_preferences(self):
        """Sample random preferences using Dirichlet distribution"""
        # Dirichlet ensures diversity and sum close to ~3
        weights = np.random.dirichlet([1, 1, 1])
        return {
            'parking_importance': float(weights[0]),
            'time_importance': float(weights[1]),
            'distance_importance': float(weights[2])
        }
    
    def apply_costs_to_model(self, costs):
        """Apply cost parameters to model"""
        model = copy.deepcopy(self.base_model)
        
        # Apply vehicle costs
        for vehicle in model['data']['vehicles']:
            if 'definition' in vehicle:
                vehicle['definition']['costPerTravelHour'] = costs['costPerTravelHour']
                vehicle['definition']['costPerKm'] = costs['costPerKm']
        
        # Apply parking multiplier to delivery stops
        for job in model['data']['jobs']:
            for stop in job.get('stops', []):
                if (stop.get('type') == 'SHIPMENT_DELIVERY' and 
                    'parking' in stop and 
                    'cost' in stop['parking'] and 
                    stop['parking']['cost'] > 0):
                    original_cost = stop['parking']['cost']
                    stop['parking']['cost'] = original_cost * costs['parking_multiplier']
        
        return model
    
    def send_to_odl(self, model, model_id):
        """Send model to ODL API"""
        url = f"{self.base_url}/models/{model_id}"
        try:
            response = self.session.put(url, json=model, timeout=30)
            response.raise_for_status()
            return True
        except Exception as e:
            print(f"  ✗ Upload failed: {e}")
            return False
    
    def get_plan(self, model_id):
        """Retrieve plan from ODL API"""
        url = f"{self.base_url}/models/{model_id}/optimiserstate/plan"
        try:
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"  ✗ Get plan failed: {e}")
            return None
    
    def extract_features(self, plan):
        """Extract route features from plan"""
        if not plan or 'vehiclePlans' not in plan:
            return None
        
        features = {
            'total_distance_km': 0.0,
            'total_travel_hours': 0.0,
            'total_cost': 0.0,
            'num_stops': 0,
            'num_vehicles_used': 0,
            'unplanned_jobs': len(plan.get('unplannedJobs', [])),
            'avg_parking_difficulty': 0.0
        }
        
        total_parking_cost = 0.0
        total_stops = 0
        
        for vehicle_plan in plan['vehiclePlans']:
            stops = vehicle_plan.get('plannedStops', [])
            
            if len(stops) > 0:
                features['num_vehicles_used'] += 1
                features['num_stops'] += len(stops)
                total_stops += len(stops)
            
            time_stats = vehicle_plan.get('timeStatistics', {})
            features['total_distance_km'] += time_stats.get('travelMetres', 0) / 1000.0
            features['total_travel_hours'] += time_stats.get('travelSeconds', 0) / 3600.0
            features['total_cost'] += time_stats.get('cost', 0)
            
            breakdown = time_stats.get('optimiserCostBreakdown', {})
            total_parking_cost += breakdown.get('PARKING_COST', 0.0)
        
        if total_stops > 0:
            features['avg_parking_difficulty'] = total_parking_cost / total_stops
        
        # Round
        features['total_distance_km'] = round(features['total_distance_km'], 2)
        features['total_travel_hours'] = round(features['total_travel_hours'], 2)
        features['total_cost'] = round(features['total_cost'], 2)
        features['avg_parking_difficulty'] = round(features['avg_parking_difficulty'], 2)
        
        return features
    
    # ========================================================================
    # PHASE 1: BUILD COST POOL
    # ========================================================================
    
    def build_cost_pool(self, pool_size, wait_seconds, checkpoint_every, pool_file):
        """Phase 1: Generate pool of cost sets with optimized solutions"""
        print(f"\n{'='*60}")
        print(f"PHASE 1: BUILDING COST POOL")
        print(f"{'='*60}")
        print(f"Target: {pool_size} cost sets")
        print(f"Estimated time: {(pool_size * wait_seconds) / 3600:.1f} hours")
        print(f"{'='*60}\n")
        
        start_time = time.time()
        
        try:
            for i in range(1, pool_size + 1):
                print(f"\nCost Set {i}/{pool_size}:")
                
                try:
                    costs = self.sample_random_costs()
                    print(f"  Costs: {costs['costPerTravelHour']:.2f}/h, "
                          f"{costs['costPerKm']:.4f}/km, parking×{costs['parking_multiplier']:.2f}")
                    
                    model = self.apply_costs_to_model(costs)
                    model_id = f"cost_pool_{i}"
                    
                    if not self.send_to_odl(model, model_id):
                        continue
                    
                    print(f"  ✓ Uploaded, waiting {wait_seconds}s...")
                    time.sleep(wait_seconds)
                    
                    plan = self.get_plan(model_id)
                    features = self.extract_features(plan)
                    
                    if features is None:
                        print(f"  ✗ Failed to extract features")
                        continue
                    
                    print(f"  Features: {features['total_distance_km']:.1f}km, "
                          f"{features['total_travel_hours']:.1f}h, "
                          f"parking={features['avg_parking_difficulty']:.1f}, "
                          f"unplanned={features['unplanned_jobs']}")
                    
                    self.cost_pool.append({
                        'pool_id': i,
                        'costs': costs,
                        'features': features,
                        'timestamp': datetime.now().isoformat()
                    })
                    
                    print(f"  ✓ Added (total: {len(self.cost_pool)})")
                    
                    if i % checkpoint_every == 0:
                        self.save_cost_pool(pool_file)
                        elapsed = time.time() - start_time
                        remaining = ((pool_size - i) * elapsed / i)
                        print(f"\n  CHECKPOINT: {len(self.cost_pool)}/{pool_size} | "
                              f"Elapsed: {elapsed/3600:.1f}h | Remaining: {remaining/3600:.1f}h\n")
                
                except Exception as e:
                    print(f"  ✗ Error: {e}")
        
        except KeyboardInterrupt:
            print(f"\n\nInterrupted! Pool has {len(self.cost_pool)} entries")
        
        self.save_cost_pool(pool_file)
        
        elapsed = time.time() - start_time
        print(f"\n{'='*60}")
        print(f"PHASE 1 COMPLETE")
        print(f"{'='*60}")
        print(f"Cost pool: {len(self.cost_pool)} solutions")
        print(f"Time: {elapsed/3600:.1f} hours")
        print(f"Saved to: {pool_file}")
        print(f"{'='*60}\n")
    
    def save_cost_pool(self, pool_file):
        """Save cost pool to file"""
        with open(pool_file, 'w') as f:
            json.dump(self.cost_pool, f, indent=2)
        print(f"  💾 Saved to {pool_file}")
    
    def load_cost_pool(self, pool_file):
        """Load cost pool from file"""
        try:
            with open(pool_file, 'r') as f:
                self.cost_pool = json.load(f)
            print(f"✓ Loaded {len(self.cost_pool)} solutions from {pool_file}")
            return True
        except FileNotFoundError:
            print(f"✗ File not found: {pool_file}")
            return False
    
    # ========================================================================
    # PHASE 2: MATCH PREFERENCES TO POOL (PARETO TOP-N)
    # ========================================================================
    
    def find_best_match(self, preferences, top_n=30):
        """
        Find best match using Pareto Top-N selection
        
        Process:
        1. Get top N solutions (closest to ideal point)
        2. Select best using weighted Pareto evaluation
        """
        if len(self.cost_pool) == 0:
            return None
        
        analyzer = ParetoAnalyzer(self.cost_pool)
        top_solutions = analyzer.get_top_n_solutions(n=top_n)
        best_match = analyzer.select_best(top_solutions, preferences)
        
        return best_match
    
    def generate_training_samples(self, n_samples, top_n, output_file):
        """Phase 2: Generate training samples using Pareto Top-N"""
        if len(self.cost_pool) == 0:
            print("✗ Cost pool is empty! Load or build first.")
            return
        
        print(f"\n{'='*60}")
        print(f"PHASE 2: GENERATING TRAINING SAMPLES")
        print(f"{'='*60}")
        print(f"Cost pool: {len(self.cost_pool)} solutions")
        print(f"Target samples: {n_samples}")
        print(f"Top-N selection: {top_n}")
        print(f"{'='*60}\n")
        
        self.training_data = []
        
        for i in range(1, n_samples + 1):
            print(f"\rSample {i}/{n_samples}", end='', flush=True)
            
            preferences = self.sample_preferences()
            match = self.find_best_match(preferences, top_n=top_n)
            
            if match is None:
                continue
            
            sample = {
                'sample_id': i,
                'preferences': preferences,
                'costs': match['costs'],
                'features': match['features'],
                'metadata': {
                    'pool_id': match['pool_id'],
                    'pareto_distance': match['pareto_distance'],
                    'selection_pool_size': match['selection_pool_size'],
                    'timestamp': datetime.now().isoformat()
                }
            }
            
            self.training_data.append(sample)
        
        # Save
        with open(output_file, 'w') as f:
            json.dump(self.training_data, f, indent=2)
        
        print(f"\n\n{'='*60}")
        print(f"PHASE 2 COMPLETE")
        print(f"{'='*60}")
        print(f"Generated: {len(self.training_data)} samples")
        print(f"Saved to: {output_file}")
        
        # Statistics
        pool_ids = [s['metadata']['pool_id'] for s in self.training_data]
        unique_ids = len(set(pool_ids))
        print(f"\nDiversity: {unique_ids} unique solutions used")
        print(f"Avg samples per solution: {len(self.training_data) / unique_ids:.1f}")
        print(f"{'='*60}\n")

print("✓ CostPoolDataCollector class defined")

✓ CostPoolDataCollector class defined


## 5. Initialize Collector

In [5]:
collector = CostPoolDataCollector(
    base_url=BASE_URL,
    username=USERNAME,
    password=PASSWORD,
    model_path=MODEL_PATH,
    cost_ranges=COST_RANGES
)

✓ Initialized
  Jobs: 106
  Vehicles: 26


## 6. Phase 1: Build Cost Pool (or Load Existing)

**If you already have cost_pool.json, skip to Phase 2!**

In [6]:
# Option A: Load existing pool
collector.load_cost_pool(POOL_FILE)

✓ Loaded 200 solutions from cost_pool.json


True

In [ ]:
# Option B: Build new pool (takes ~6 hours for 200 samples at 30s each)
collector.build_cost_pool(
    pool_size=POOL_SIZE,
    wait_seconds=WAIT_SECONDS,
    checkpoint_every=CHECKPOINT_EVERY,
    pool_file=POOL_FILE
)

## 7. Analyze Cost Pool (Optional)

In [7]:
# Quick analysis
if len(collector.cost_pool) > 0:
    analyzer = ParetoAnalyzer(collector.cost_pool)
    top_30 = analyzer.get_top_n_solutions(n=30)
    
    print(f"Cost pool analysis:")
    print(f"  Total solutions: {len(collector.cost_pool)}")
    print(f"  Top 30 solutions: {len(top_30)}")
    print(f"  Top 30 = top {100*len(top_30)/len(collector.cost_pool):.1f}%")
    
    # Check unplanned jobs
    unplanned = [s['features']['unplanned_jobs'] for s in collector.cost_pool]
    print(f"\n  Unplanned jobs: min={min(unplanned)}, max={max(unplanned)}, avg={np.mean(unplanned):.1f}")
    if max(unplanned) > 5:
        print(f"  ⚠️  Consider using relaxed model (remove skill requirements)")

Cost pool analysis:
  Total solutions: 200
  Top 30 solutions: 30
  Top 30 = top 15.0%

  Unplanned jobs: min=20, max=20, avg=20.0
  ⚠️  Consider using relaxed model (remove skill requirements)


## 8. Phase 2: Generate Training Samples

In [8]:
# Generate training data using Pareto Top-N
collector.generate_training_samples(
    n_samples=N_SAMPLES,
    top_n=TOP_N,
    output_file=OUTPUT_FILE
)


PHASE 2: GENERATING TRAINING SAMPLES
Cost pool: 200 solutions
Target samples: 400
Top-N selection: 30

Sample 400/400

PHASE 2 COMPLETE
Generated: 400 samples
Saved to: training_data_v4_pareto.json

Diversity: 5 unique solutions used
Avg samples per solution: 80.0



## 9. View Sample

In [9]:
# Show first training sample
if len(collector.training_data) > 0:
    print(json.dumps(collector.training_data[0], indent=2))

{
  "sample_id": 1,
  "preferences": {
    "parking_importance": 0.2211968236503381,
    "time_importance": 0.19164522938840747,
    "distance_importance": 0.5871579469612545
  },
  "costs": {
    "costPerTravelHour": 1.23,
    "costPerKm": 0.3214,
    "parking_multiplier": 0.42
  },
  "features": {
    "total_distance_km": 1001.33,
    "total_travel_hours": 30.4,
    "total_cost": 450131.92,
    "num_stops": 172,
    "num_vehicles_used": 8,
    "unplanned_jobs": 20,
    "avg_parking_difficulty": 87.99
  },
  "metadata": {
    "pool_id": 4,
    "pareto_distance": 0.15907641614065898,
    "selection_pool_size": 30,
    "timestamp": "2025-10-29T10:04:45.595817"
  }
}


## Done! 🎉

You now have training data with:
- ~30 unique cost combinations (from top 15% of pool)
- 400 samples total
- ~13 samples per unique output
- Ready for regression training!